In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import torch
from torch.utils.data import Dataset, DataLoader

from torch import nn
from torch import optim

import matplotlib.pyplot as plt

from numpy import vstack
from sklearn.metrics import accuracy_score

In [ ]:
heart = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
# defining X and Y
X = heart[['age','sex','chol','fbs','trtbps']]
Y = heart[['output']]

# it seems there is a lot of difference between X columns
X = preprocessing.StandardScaler().fit(X).transform(X)

# splitting train and test data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

In [ ]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
# Convert data to torch tensors
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [ ]:
batch_size = 64

# Instantiate training and test data
train_data = Data(X_train, y_train)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(X_test, y_test)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [ ]:
input_dim = 5
hidden_dim = 20
output_dim = 1

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        nn.init.kaiming_uniform_(self.layer_1.weight, nonlinearity="relu")
        self.layer_2 = nn.Linear(hidden_dim, output_dim)
       
    def forward(self, x):
        x = torch.nn.functional.relu(self.layer_1(x))
        x = torch.nn.functional.sigmoid(self.layer_2(x))

        return x
    
model = NeuralNetwork(input_dim, hidden_dim, output_dim)

In [ ]:
criterion=nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
num_epochs = 1000
loss_values = []


for epoch in range(num_epochs):
    for X, y in train_dataloader:
        # zero the parameter gradients
        optimizer.zero_grad()
       
        # forward + backward + optimize
        pred = model(X)
        loss = criterion(pred, y)
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()

In [ ]:
step = np.linspace(0, 1000, 4000)

fig, ax = plt.subplots(figsize=(8,5))
plt.plot(step, np.array(loss_values))
plt.title("Step-wise Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
predictions, actuals = list(), list()
with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_dataloader):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        y_pred = yhat.round()
        # store
        predictions.append(y_pred)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    
print(acc)